Run this notebook to reduce data from the Nickel Telescope. Be sure to read the notes in between the cells.

In [1]:
import numpy as np
import pandas as pd
import os
from astropy.io import fits
from pathlib import Path

from overscan_subtraction import overscan_subtraction
from bias_subtraction import bias_subtraction
from dark_subtraction import dark_subtraction
from flat_division import flat_division
from correct_object_name import correct_object_name

The reduction requires the OBJECT names in the raw FITS files (which are set at the time of the observation) to be one of the following:
- "bias"
- "dark"
- "dome flat"
- "sky flat" (i.e., flats taken of the sky at sunset)
- "focus"
- your target names (can be anything)

If you need to correct OBJECT an in FITS headers of any files, do that below. If everything is correct, ignore the next cell.

In [2]:
# correct_object(myfiles, "dark")

# # Set object type names
bias_object = 'Bias'
dome_flat_object = 'Dome flat'
sky_flat_object = 'Flat'
dark_object = 'dark'
focus_object = 'focus'

# bias_object = 'bias'
# dome_flat_object = 'flat'
# sky_flat_object = 'sky flat'
# dark_object = 'dark'
# focus_object = 'focus'


Now you're ready to reduce your data. First, get a list of all the raw data files. The name of the directory with the raw data should have format 'YYYY-MM-DD-nickel-raw'.

In [3]:
# rawdir = Path("C:/Users/allis/Documents/2024-2025_Local/Akamai Internship/pipeline-testing/2024-05-12/raw")  # path to directory with raw data
# rawdir = Path("C:/Users/allis/Documents/2024-2025_Local/Akamai Internship/pipeline-testing/test-data-05-12/raw")  # path to directory with raw data
# rawdir = Path("C:/Users/allis/Documents/2024-2025_Local/Akamai Internship/pipeline-testing/test-data-06-24/raw")  # path to directory with raw data
rawdir = Path("C:/Users/allis/Documents/2024-2025_Local/Akamai Internship/pipeline-testing/test-data-06-26/raw")
rawfiles = [file for file in rawdir.iterdir() if file.is_file()]

# # Deletes bad flats from the 05-12-24 data run
# rawfiles = rawfiles[:36] + rawfiles[41:]

# Deletes bad files from the 06-26-24 data run
del rawfiles[115]
del rawfiles[112:114]
del rawfiles[108:110]
del rawfiles[74:78]
del rawfiles[34]




Create processing and reduced directories.

In [4]:
rootdir = rawdir.parent
procdir = rawdir.parent / (rawdir.name + "-proc")
reddir = rawdir.parent / (rawdir.name + "-reduced")

Path.mkdir(procdir, exist_ok=True)
Path.mkdir(reddir, exist_ok=True)

Do overscan subtraction.

In [5]:
overscan_dir = procdir / 'overscan'
print(overscan_dir)
Path.mkdir(overscan_dir, exist_ok=True)
overscan_files = [overscan_dir / (file.stem + file.suffix) for file in rawfiles]

overscan_subtraction(rawfiles, overscan_files, 'yes')

C:\Users\allis\Documents\2024-2025_Local\Akamai Internship\pipeline-testing\test-data-06-26\raw-proc\overscan


Make a dataframe of all the files we want to continue reducing.

In [6]:
obj_list = []
exptime_list = []
filt_list = []

for overscan_file in overscan_files:
    hdul = fits.open(str(overscan_file))
    obj_list.append(hdul[0].header["OBJECT"])
    exptime_list.append(hdul[0].header["EXPTIME"])
    filt_list.append(hdul[0].header["FILTNAM"])
    hdul.close()

df_log = pd.DataFrame({
    "file": overscan_files,
    "object": obj_list,
    "exptime": exptime_list,
    "filt": filt_list
    })
df_log

,file,object,exptime,filt
0,C:\Users\allis\Documents\2024-2025_Local\Akama...,test,0.0,6563/100
1,C:\Users\allis\Documents\2024-2025_Local\Akama...,Bias,0.0,6563/100
2,C:\Users\allis\Documents\2024-2025_Local\Akama...,Bias,0.0,6563/100
3,C:\Users\allis\Documents\2024-2025_Local\Akama...,Bias,0.0,6563/100
4,C:\Users\allis\Documents\2024-2025_Local\Akama...,Bias,0.0,6563/100
...,...,...,...,...
107,C:\Users\allis\Documents\2024-2025_Local\Akama...,Flat,10.0,B
108,C:\Users\allis\Documents\2024-2025_Local\Akama...,Flat,10.0,B
109,C:\Users\allis\Documents\2024-2025_Local\Akama...,Flat,10.0,B
110,C:\Users\allis\Documents\2024-2025_Local\Akama...,Flat,10.0,B


Do bias subtraction.

In [7]:
# gather all the bias frames
biasfiles = list(df_log.file[df_log.object == bias_object])

# average all of them into one
biasdata = []
for biasfile in biasfiles:
    hdul = fits.open(str(biasfile))
    biasdata.append(hdul[0].data)
    hdul.close()
bias = np.stack(biasdata).mean(axis=0)

# omit hot column so that it is properly flat-fielded out
# Allison Note: column is saturated to around 65,000 cts in all images; unusable (???)
bias[:,256] = 0

# gather all non-bias files in a subdirectory of -proc
nonbias_dir = procdir / 'unbias'
print(nonbias_dir)
Path.mkdir(nonbias_dir, exist_ok=True)

nonbias_files_input = list(df_log.file[df_log.object != bias_object])
nonbias_files_output = [nonbias_dir / (file.stem.split('_')[0] + file.suffix) for file in nonbias_files_input]

bias_subtraction(nonbias_files_input, nonbias_files_output, bias)

df_log["file"] = [nonbias_dir / (file.stem.split('_')[0] + file.suffix) for file in df_log["file"]]
df_log

C:\Users\allis\Documents\2024-2025_Local\Akamai Internship\pipeline-testing\test-data-06-26\raw-proc\unbias


,file,object,exptime,filt
0,C:\Users\allis\Documents\2024-2025_Local\Akama...,test,0.0,6563/100
1,C:\Users\allis\Documents\2024-2025_Local\Akama...,Bias,0.0,6563/100
2,C:\Users\allis\Documents\2024-2025_Local\Akama...,Bias,0.0,6563/100
3,C:\Users\allis\Documents\2024-2025_Local\Akama...,Bias,0.0,6563/100
4,C:\Users\allis\Documents\2024-2025_Local\Akama...,Bias,0.0,6563/100
...,...,...,...,...
107,C:\Users\allis\Documents\2024-2025_Local\Akama...,Flat,10.0,B
108,C:\Users\allis\Documents\2024-2025_Local\Akama...,Flat,10.0,B
109,C:\Users\allis\Documents\2024-2025_Local\Akama...,Flat,10.0,B
110,C:\Users\allis\Documents\2024-2025_Local\Akama...,Flat,10.0,B


Do dark subtraction.

This can usually be skipped, since the Nickel CCD has a very low dark current, but it is included here for the sake of completeness.

In [8]:
# Code deleted

Do flat division.

Divide each pixel and then multiply all pixels by the average of the flat frame.

If sky (sunset) flats are available, those are used. If they are not available, dome flats are used.

In [9]:
# use sky flats if available, use dome flats if not
if sky_flat_object in list(set(obj_list)):
    flattype = sky_flat_object
else:
    flattype = dome_flat_object
    
flatfilts = list(set(df_log.filt[df_log.object == flattype]))

for flatfilt in flatfilts:
    # find all the files with this filter
    flatfiles = list(df_log.file[(df_log.object == flattype) & (df_log.filt == flatfilt)])
    scienceobjects = list(set(df_log.object[(df_log.object != bias_object) &
                                            (df_log.object != dark_object) &
                                            (df_log.object != dome_flat_object) &    # modified from 'dome flat' to 'flat' for 05-12-24 data
                                            (df_log.object != sky_flat_object) &
                                            (df_log.object != focus_object) &
                                            (df_log.filt == flatfilt)]))
    
    # calculate the average flat frame
    if len(flatfiles) > 1:
        flatdata = []
        for flatfile in flatfiles:
            hdul = fits.open(str(flatfile))
            flatdata.append(hdul[0].data)
            hdul.close()
        flat = np.stack(flatdata).mean(axis=0)
    else:
        hdul = fits.open(str(flatfiles[0]))
        flat = hdul[0].data
        hdul.close()
        
    if len(scienceobjects) > 0:
        for scienceobject in scienceobjects:
            sciencefiles = list(df_log.file[(df_log.object == scienceobject) &
                                            (df_log.filt == flatfilt)])
            
            # make a new directory for each science target / filter combination
            sci_dir = reddir / (scienceobject + '_' + flatfilt)
            Path.mkdir(sci_dir, exist_ok=True)

            # define reduced file names
            redfiles = [sci_dir / (file.stem.split('_')[0] + '_red' + file.suffix) for file in sciencefiles]

            # do flat division
            if len(sciencefiles) > 0:
                flat_division(sciencefiles, redfiles, flat)

You're done! Your reduced images are now ready for your viewing.

Delete all overscan subtracted files to save memory

In [10]:
# for file in overscan_files:
#     file.unlink()

Use the code below to go back through and check values in the FITS files (- Allison)

In [11]:
# # Examining average column value of hot column 257 -- data unusable in this column; saturated pixels
# for image in rawfiles:
#     with fits.open(image, mode='update') as hdu:
#         data = hdu[0].data
#         header = hdu[0].header
#         impath = header["FITSFILE"]
#         objtype = header["OBJECT"]
#         print(f"Image {impath} (type {objtype}) has avg value in hot column 257 = {np.mean(data[:,256]):.1f}")

# # Examining hot column 1003 vs regular column 1000
# # Inconsistent ratio and difference (uncorrelated w/ exposure time?)
# for image in rawfiles:
#     with fits.open(image, mode='update') as hdu:
#         data = hdu[0].data
#         header = hdu[0].header
#         # impath = header["FITSFILE"]
#         objtype = header["OBJECT"]
#         exptime = header["EXPTIME"]
#         print(f"Image {image.name} ({objtype}, {exptime}): col 1003 avg = {np.mean(data[:,1002]):.1f}, col 1000 avg = {np.mean(data[:,999]):.1f}. diff = {np.mean(data[:,784])-np.mean(data[:,783]):.3f}, ratio = {np.mean(data[:,1002])/np.mean(data[:,999]):.2f}")

# # Examining difference between dull column 784 and bright column 785
# for image in rawfiles:
#     with fits.open(image, mode='update') as hdu:
#         data = hdu[0].data
#         header = hdu[0].header
#         # impath = header["FITSFILE"]
#         objtype = header["OBJECT"]
#         exptime = header["EXPTIME"]
#         print(f"Image {image.name} ({objtype}, {exptime}): col 784 avg = {np.mean(data[:,783]):.1f}, col 785 avg = {np.mean(data[:,784]):.1f}. diff = {np.mean(data[:,784])-np.mean(data[:,783]):.3f}, ratio = {np.mean(data[:,784])/np.mean(data[:,783]):.2f}")

# Make sure that all filter names are correct
# for sci_image in rawfiles[44:]:
#     with fits.open(sci_image, mode='update') as hdu:
#         header = hdu[0].header
#         header["FILTNAM"] = "V"
#         hdu.flush()

# with fits.open("C:/Users/allis/Documents/2024-2025_Local/Akamai Internship/pipeline-testing/test-data-05-12/raw/d1076.fits", mode='update') as hdu:
#     header = hdu[0].header
#     header["FILTNAM"] = "I"
#     hdu.flush()

# # Display many files in a row
# import matplotlib.pyplot as plt
# display_dir = Path("C:/Users/allis/Documents/2024-2025_Local/Akamai Internship/pipeline-testing/test-data-05-12/raw-reduced/sky flat_V")
# for image in display_dir.iterdir():
#     with fits.open(image) as hdul:
#         # print("HDU List Info:")
#         # print(hdul.info())
#         # print("\nHDU Header")
#         # print(repr(hdul[0].header))
        
#         # print(hdul[0].data)
        
#         plt.figure(figsize=(4, 3))  # Set the figure size if needed
#         plt.imshow(hdul[0].data, origin='lower')
#         # Set the DPI (dots per inch) for the figure
#         plt.gcf().set_dpi(300)  # Adjust the DPI value as needed

#         # Display the plot
#         plt.colorbar()
#         plt.show()
